# Pipe Friction Loss Calc

### Import Libraries, Format, and Set Working Directory

In [2]:
import numpy as np
import pandas as pd
import sys
import os
import xlwings as xw
import matplotlib as plt
import pint
import openpyxl
from openpyxl import load_workbook

In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
os.chdir(r"E:\Python\Calculators\Pipe Calc")

### Load Pipe and Fitting Data

In [4]:
df_diam=pd.read_excel('Fittings and Diam.xlsx',sheet_name='TYPE', index_col=[0], header=[0])
df_diam.rename_axis(index=["Nom Diam"],inplace=True)
df_fitting_thrd=pd.read_excel('Fittings and Diam.xlsx',sheet_name='THREAD', index_col=[0], header=[0])
df_fitting_thrd.rename_axis(index=["Nom Diam"], inplace=True)
df_fitting_flngd=pd.read_excel('Fittings and Diam.xlsx',sheet_name='FLANGE OR WELD', index_col=[0], header=[0])
df_fitting_flngd.rename_axis(index=["Nom Diam"], inplace=True)
print(df_diam)
df_spec_rough=pd.read_excel('Fittings and Diam.xlsx',sheet_name='ROUGH', index_col=[0], header=[0])
df_spec_rough
print(df_fitting_thrd)
print(df_fitting_flngd)

          STEEL SCH 40  COPPER TYPE L
Nom Diam                             
0.50             0.607          0.545
0.75             0.824          0.785
1.00             1.049          1.025
1.25             1.380          1.265
1.50             1.610          1.505
2.00             2.067          1.985
2.50             2.469          2.465
3.00             3.068          2.945
3.50             3.548          3.425
4.00             4.026          3.905
4.50             4.506            NaN
5.00             5.047          4.875
6.00             6.065          5.845
7.00             7.023            NaN
8.00             7.981          7.725
9.00             8.941            NaN
10.00           10.020          9.625
12.00           12.000         11.565
14.00           13.250            NaN
          90° Standard Elbow 90° Long Radius Elbow  45° Elbow  Return Bend  \
Nom Diam                                                                     
0.375                   2.50                  

### User Inputs

In [5]:
Q=float(input('Enter fluid flowrate [GPM] <<=  '))
diam_nom=float(input('Enter nominal pipe diameter (decimal size) <<=  '))
pipe_matl=str(input('Enter pipe material type [c=copper, s=steel p=PVC] <<=  '))
FluidTemp=float(input('Enter approximate fluid temperature <<=  '))
pipe_lngth=float(input('Enter pipe length [ft] <<=  ') or '100')
fitting_type=str(input('Type of pipe fittings? [t=threaded, f=flanged or welded] <==  ') or 'f')

Enter fluid flowrate [GPM] <<=   12000
Enter nominal pipe diameter (decimal size) <<=   3
Enter pipe material type [c=copper, s=steel p=PVC] <<=   s
Enter approximate fluid temperature <<=   90
Enter pipe length [ft] <<=   100
Type of pipe fittings? [t=threaded, f=flanged or welded] <==   t


### Determination of Pipe I.D. and Specific Roughness

In [6]:
if pipe_matl=='c' or pipe_matl=='C':
    d_hyd=df_diam.loc[diam_nom, 'COPPER TYPE L']
    spec_rough=df_spec_rough.loc['COPPER TYPE L']
if pipe_matl=='s' or pipe_matl=='S':
    d_hyd=df_diam.loc[diam_nom, 'STEEL SCH 40']
    spec_rough=float(df_spec_rough.loc['STEEL SCH 40'])

print('hydralic diameter [in] =>>  ', d_hyd)
print('Specific Roughness =>>  ', spec_rough)

hydralic diameter [in] =>>   3.068
Specific Roughness =>>   0.0002


### Calculate Velocity and Relative Roughness

In [7]:
d_hyd_ft=d_hyd/12
V=(0.408*Q)/((d_hyd)**2)
print('Velocity [ft/s] =>>  ',V)
visc=np.array([[40,0.00001664],[50,0.0000141],[60,0.00001217],[70,0.00001059],[80,0.0000093],[90,0.00000826],[100,0.00000739],[110,0.00000667],[120,0.00000609],[130,0.00000558],[140,0.00000514]])
xp=visc[:,0]
fp=visc[:,1]
visc_k=np.interp(FluidTemp,xp,fp)
rel_rough=spec_rough/(d_hyd_ft)
print('Relative Roughness =>>  ', rel_rough)

Velocity [ft/s] =>>   520.15251007583
Relative Roughness =>>   0.0007822685788787485


### Calculate Reynolds Numer (Now w/ a user defined function)

In [8]:
def Reynolds(V,d_hyd_ft,visc_k):
    Re=V*d_hyd_ft/visc_k
    return Re
Re=Reynolds(V,d_hyd_ft,visc_k)
print('Reynolds Number =>>  ',"{:,.6}".format(Re))

Reynolds Number =>>   1.61e+07


### Calculate Friction Factor (Now w/ a user defined function)

In [9]:
def fric_f(Re,rel_rough):
    ff=0.25/((np.log10(rel_rough/3.7)+(5.74/(Re**0.9))))**2
    return ff
ff=fric_f(Re,rel_rough)
print('Friction Factor =>>  ',"{:,.4}".format(ff))

Friction Factor =>>   0.01851


### Calculate "Major" Fiction Loss (Now w/ a user defined function)

In [15]:
def Hf_major(fric_f,pipe_lngth,d_hyd_ft,V):
    Hf_major=ff*pipe_lngth/(d_hyd_ft)*(V**2/(2*32.2))
    return Hf_major
dP_major=Hf_major(fric_f,pipe_lngth,d_hyd_ft,V)
dP_major

30420.30909360075

In [16]:
fitting_num=np.zeros(20)
fitting_k=np.zeros(20)
fittings=['90° Elbows - Short Radius','90° Elbows - Long Radius','Butterfly Valves','Reducers','Expansions','Balancing Valves','Tees - Line Flow','Tees - Branch Flow','45° Elbow','Other']
for idx, word in enumerate(fittings):
    print(word)
    fitting_num[idx]=(input('Enter Number of Fittings from Above :  ') or 0)
    if fitting_num[idx]!=0:
        fitting_k[idx]=(input('Enter K-Factor for Fitting :  ') or 0)
		
		
K_total=fitting_num*fitting_k
K_sumTotal=K_total.sum()
Hf_minor=K_sumTotal*((V**2)/(2*32.2))


print('Fitting Losses [ft] =>>  ',Hf_minor)
print('Straight Pipe Losses [ft] =>>  ',Hf_major)

other_new=1
other=0
while other_new!=0:
    other1=other
    other_new=float(input('Enter Additional dP losses? If so, enter total dP [ft] <<=  ') or 0)
    other=other1+other_new
Hf_other=other
print('Head loss from other components =>>  ', Hf_other)

dp_ft=Hf_minor+dP_major+Hf_other
print('Total Pressure Drop in Feet =>>  ', dp_ft)
print('Flow Rate [GPM] =>>  ', Q)

90° Elbows - Short Radius


Enter Number of Fittings from Above :   


90° Elbows - Long Radius


Enter Number of Fittings from Above :   


Butterfly Valves


...

Other


Enter Number of Fittings from Above :   


Fitting Losses [ft] =>>   0.0
Straight Pipe Losses [ft] =>>   <function Hf_major at 0x0000019633547670>


Enter Additional dP losses? If so, enter total dP [ft] <<=   


Head loss from other components =>>   0.0
Total Pressure Drop in Feet =>>   30420.30909360075
Flow Rate [GPM] =>>   12000.0
